In [19]:
!pip install pyspark

In [20]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Ejemplo de PySpark en Jupyter Notebook") \
    .getOrCreate()

# Obtener el SparkContext
sc = spark.sparkContext

In [21]:
graph = [(1,11,2),(1,11,3),(2,11,3),(3,11,2),(3,11,4),(4,11,1),(4,11,2),(4,11,3),(4,12,5),(5,12,1),(5,12,2),(5,12,6)]
#graph = [(2,11,2), (1,11,2), (2,11,1)]

In [22]:
B = 2

In [23]:
def hash(x):
    return x % B

In [24]:
def map_pdm(x, arista, y):

    x_hash = hash(x)
    y_hash = hash(y)

    l = []
    for i in range(B):
        l.append(((x_hash, y_hash, i), (x, arista, y)))
        l.append(((i, x_hash, y_hash), (x, arista, y)))
        l.append(((y_hash, i, x_hash), (x, arista, y)))

    return set(l)

In [25]:
datos_rdd = sc.parallelize(graph)

### Pasarlo por map

In [26]:
resultado_rdd = datos_rdd.flatMap(lambda dato: map_pdm(*dato))

resultado_rdd.collect()

[((1, 0, 0), (1, 11, 2)),
 ((0, 1, 1), (1, 11, 2)),
 ((1, 0, 1), (1, 11, 2)),
 ((0, 1, 0), (1, 11, 2)),
 ((0, 0, 1), (1, 11, 2)),
 ((1, 1, 0), (1, 11, 2)),
 ((1, 1, 1), (1, 11, 3)),
 ((1, 1, 0), (1, 11, 3)),
 ((1, 0, 1), (1, 11, 3)),
 ((0, 1, 1), (1, 11, 3)),
 ((1, 0, 1), (2, 11, 3)),
 ((0, 0, 1), (2, 11, 3)),
 ((0, 1, 0), (2, 11, 3)),
 ((1, 1, 0), (2, 11, 3)),
 ((1, 0, 0), (2, 11, 3)),
 ((0, 1, 1), (2, 11, 3)),
 ((0, 0, 1), (3, 11, 2)),
 ((1, 1, 0), (3, 11, 2)),
 ((1, 0, 0), (3, 11, 2)),
 ((0, 1, 1), (3, 11, 2)),
 ((1, 0, 1), (3, 11, 2)),
 ((0, 1, 0), (3, 11, 2)),
 ((1, 1, 0), (3, 11, 4)),
 ((1, 0, 0), (3, 11, 4)),
 ((0, 1, 1), (3, 11, 4)),
 ((1, 0, 1), (3, 11, 4)),
 ((0, 0, 1), (3, 11, 4)),
 ((0, 1, 0), (3, 11, 4)),
 ((1, 0, 1), (4, 11, 1)),
 ((0, 0, 1), (4, 11, 1)),
 ((0, 1, 0), (4, 11, 1)),
 ((1, 1, 0), (4, 11, 1)),
 ((1, 0, 0), (4, 11, 1)),
 ((0, 1, 1), (4, 11, 1)),
 ((0, 0, 0), (4, 11, 2)),
 ((1, 0, 0), (4, 11, 2)),
 ((0, 0, 1), (4, 11, 2)),
 ((0, 1, 0), (4, 11, 2)),
 ((1, 0, 1),

### Reduce


In [39]:
test = resultado_rdd.reduceByKey(lambda x, y: x + y)
test.collect()[0]

((1, 0, 0),
 (1,
  11,
  2,
  2,
  11,
  3,
  3,
  11,
  2,
  3,
  11,
  4,
  4,
  11,
  1,
  4,
  11,
  2,
  4,
  11,
  3,
  4,
  12,
  5,
  5,
  12,
  2,
  5,
  12,
  6))

In [40]:
def triangulo(nodos):

  cont = 0
  posibles = []
  trio = []

  for i in nodos:

    trio.append(i)
    cont += 1

    if cont % 3 == 0:
      posibles.append(tuple(trio))
      trio = []

  print(posibles)
  final = []
  for i in range(len(posibles)):
    trio_2 = posibles[i]

    for j in range(i, len(posibles)):
      if j + 1 >= len(posibles):
        break
      sig = posibles[j]
      sub_sig = posibles[j + 1]


      if trio_2[2] == sig[0] and sig[2] == sub_sig[0] and trio_2[0] == sub_sig[2]:
        final.append((trio_2[0], sig[0], sub_sig[0]))

      elif trio_2[2] == sub_sig[0] and sig[0] == sub_sig[2] and trio_2[0] == sig[2]:
        final.append((trio_2[0], sig[0], sub_sig[0]))

  return final

In [41]:
dicc = {}
for i in test.collect():
  dicc[i[0]] = triangulo(i[1])

dicc.keys()

[(1, 11, 2), (2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 1), (4, 11, 2), (4, 11, 3), (4, 12, 5), (5, 12, 2), (5, 12, 6)]
[(1, 11, 2), (2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 1), (4, 11, 2), (4, 11, 3), (4, 12, 5), (5, 12, 2), (5, 12, 6)]
[(1, 11, 2), (2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 1), (4, 11, 2), (4, 11, 3), (4, 12, 5), (5, 12, 2), (5, 12, 6)]
[(1, 11, 3), (5, 12, 1)]
[(1, 11, 2), (1, 11, 3), (2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 1), (4, 11, 3), (4, 12, 5), (5, 12, 1), (5, 12, 2), (5, 12, 6)]
[(1, 11, 2), (1, 11, 3), (2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 1), (4, 11, 3), (4, 12, 5), (5, 12, 1), (5, 12, 2), (5, 12, 6)]
[(1, 11, 2), (1, 11, 3), (2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 1), (4, 11, 3), (4, 12, 5), (5, 12, 1), (5, 12, 2), (5, 12, 6)]
[(4, 11, 2)]


dict_keys([(1, 0, 0), (0, 1, 0), (0, 0, 1), (1, 1, 1), (0, 1, 1), (1, 0, 1), (1, 1, 0), (0, 0, 0)])

In [31]:
dicc.values()

dict_values([[], [], [], [], [(1, 3, 4)], [(1, 3, 4)], [(1, 3, 4)], []])